# PART A — Create Test Data

In [53]:
#Step 0 — Imports & Setup
import json
import pandas as pd
import os
import sys
from pathlib import Path

# Adjust if needed — point to your repo
REPO_DIR = Path("/Users/ywxiu/jasp-multimodal-rag")
sys.path.append(str(REPO_DIR))

from src.retrieval.retrieval import retrieve_top_k
from src.generation.generation import run_generation_pipeline  # your function


In [54]:
##Step 1 — Load Questions
import json
from pathlib import Path

QA_FILE = Path("/Users/ywxiu/jasp-multimodal-rag/data/test_QA/Qs.txt")

questions = []

with open(QA_FILE, "r") as f:
    for line in f:
        line = line.strip()

        # skip empty lines and emoji section headers
        if not line:
            continue
        if line.startswith("🧪") or line.startswith("📚") or line.startswith("⚙️") \
           or line.startswith("💡") or line.startswith("❓"):
            continue

        # Each remaining line is a question string
        qid = f"q_{len(questions)+1}"

        questions.append({
            "id": qid,
            "query": line,
            "answerable": None,            # you will fill manually
            "ground_truth_answer": None,   # fill later
            "relevant_chunk_ids": []       # fill later
        })

len(questions), questions[:10]


(14,
 [{'id': 'q_1',
   'query': 'How to import data from Excel or CSV into JASP?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_2',
   'query': 'How do I run a repeated measures ANOVA in JASP?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_3',
   'query': 'How to perform mediation or moderation analysis in JASP?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_4',
   'query': 'How to run PAIRED SAMPLES T-TEST in JASP?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_5',
   'query': 'Can JASP run Principal Component Analysis?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_6',
   'query': 'How to know if my data meets ANOVA assumptions?',
   'answerable': None,
   'ground_truth_answer': None,
   'relevant_chunk_ids': []},
  {'id': 'q_7

In [55]:
#Export Template JSON
OUT = Path("/Users/ywxiu/jasp-multimodal-rag/data/test_QA/questions_template.json")

with open(OUT, "w") as f:
    json.dump(questions, f, indent=2)

OUT


PosixPath('/Users/ywxiu/jasp-multimodal-rag/data/test_QA/questions_template.json')

# Then you open the JSON and fill:

"answerable": true/false

"ground_truth_answer": ""

"relevant_chunk_ids": ["chunk_xyz", ...]

# Help Function: fetch trunks based on chunk ID

In [56]:
import json
import os
from typing import Dict, Any, List, Optional

def load_all_json_chunks(folder_path: str) -> Dict[str, Dict[str, Any]]:
    """
    Load all JSON chunk files inside a folder and index them by their chunk_id (section_id or metadata.doc_id).
    
    Returns:
        dict: {
            chunk_id: {
                "file": filename,
                "text": text,
                "metadata": metadata
            },
            ...
        }
    """
    chunk_index = {}
    
    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue
        
        file_path = os.path.join(folder_path, filename)
        
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"Skipping {filename}: cannot load JSON ({e})")
            continue
        
        sections = data.get("sections", [])
        
        for section in sections:
            metadata = section.get("metadata", {})
            
            # The unique chunk ID may appear as:
            #   - metadata["section_id"]  (video + GitHub)
            #   - metadata["doc_id"]      (PDF)
            #   - or both
            possible_ids = []
            if "section_id" in metadata:
                possible_ids.append(metadata["section_id"])
            if "doc_id" in metadata:
                possible_ids.append(metadata["doc_id"])
            
            for cid in possible_ids:
                chunk_index[cid] = {
                    "file": filename,
                    "text": section.get("text", ""),
                    "metadata": metadata
                }

    return chunk_index


def get_chunks(chunk_index, chunk_id_list):
    return {cid: chunk_index.get(cid) for cid in chunk_id_list}

def print_trunks(trunk_dict):
    """
    Pretty-print the text content of each fetched chunk.
    trunk_dict: output of get_chunks()
    """
    for chunk_id, chunk in trunk_dict.items():
        if chunk is None:
            print(f"\n❌ Chunk not found: {chunk_id}")
            continue
        
        print("\n" + "="*80)
        print(f"📌 Chunk ID: {chunk_id}")
        print(f"📄 Source file: {chunk['file']}")
        print("-"*80)
        print(chunk["text"])   # print the full chunk text
        print("="*80 + "\n")



In [57]:
chunk_index = load_all_json_chunks("/Users/ywxiu/jasp-multimodal-rag/data/processed/chunks")

chunks= get_chunks(chunk_index, ["e87fc7cf","4488c4f1","de3adc1a",])
print_trunks(chunks)



📌 Chunk ID: e87fc7cf
📄 Source file: manual_Statistical-Analysis-in-JASP-A-guide-for-students-2025.json
--------------------------------------------------------------------------------
As with the Independent t-test, there are both parametric and non -parametric options available in
JASP. The parametric paired-samples t-test (also known as the dependent sample t -test or repeated
measures t -test) compares the means between two related groups on the same continuous,
dependent variable. For example, looking at weight loss pre- and post-10 weeks of dieting.
The paired t-statistic =
mean of the differences between group pairs
the standard error of the mean differences
With the paired t-test, the null hypothesis (H o) is that the pairwise difference between the two
groups is zero.


📌 Chunk ID: 4488c4f1
📄 Source file: manual_Statistical-Analysis-in-JASP-A-guide-for-students-2025.json
--------------------------------------------------------------------------------
Open Paired t-test.csv in 

# PART A — RETRIEVAL EVALUATION (H1a & H1b)

H1a. At least 90% of answerable queries will achieve success@k ≥ 1, such that the top-k retrieved results contain at least one relevant documentation chunk.

H1b. At least 80% of answerable queries will retrieve a relevant chunk at rank 1, demonstrating strong top-1 retrieval accuracy.


In [58]:
TEST_JSON = Path("/Users/ywxiu/jasp-multimodal-rag/data/test_QA/questions_template copy.json")

with open(TEST_JSON, "r") as f:
    test_set = json.load(f)

len(test_set)


12

In [59]:
#Step 4 — Run Retrieval for All Queries
retrieval_results = []

K = 3  # success@3
test_set=test_set

for item in test_set:
    q = item["query"]
    qid = item["id"]
    relevant_ids = set(item["relevant_chunk_ids"])

    # run retrieval only
    retrieved = retrieve_top_k(q, top_k=K)
   
    retrieved_ids = [
        c.metadata.get("doc_id")
        or c.metadata.get("section_id")
        for c in retrieved
    ]


    # evaluate
    success_at_k = any(rid in relevant_ids for rid in retrieved_ids[:K])
    top1_relevant = retrieved_ids[0] in relevant_ids if retrieved_ids else False

    retrieval_results.append({
        "id": qid,
        "query": q,
        "answerable": item["answerable"],
        "retrieved_ids": retrieved_ids,
        "success_at_k": success_at_k,
        "top1_relevant": top1_relevant
    })

retrieval_results[:2]


2025-11-18 16:10:11.776 | INFO     | src.retrieval.retrieval:retrieve_top_k:366 - 🔍 Starting retrieval pipeline for query: 'How to import data from Excel or CSV into JASP?'
2025-11-18 16:10:11.783 | INFO     | src.retrieval.retrieval:load_docs_for_bm25:197 - 📄 Loaded 16 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_RegressionLinearBayesian.json
2025-11-18 16:10:11.784 | INFO     | src.retrieval.retrieval:load_docs_for_bm25:197 - 📄 Loaded 33 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_RegressionLogistic.json
2025-11-18 16:10:11.787 | INFO     | src.retrieval.retrieval:load_docs_for_bm25:197 - 📄 Loaded 43 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/video_Introduction_to_JASP_APRaBFC2lEQ_chunks.json
2025-11-18 16:10:11.789 | INFO     | src.retrieval.retrieval:load_docs_for_bm25:197 - 📄 Loaded 58 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_MixedModelsLMM.json
2025-11-18 16:10:11.7

[{'id': 'q_1',
  'query': 'How to import data from Excel or CSV into JASP?',
  'answerable': True,
  'retrieved_ids': ['0ecf6792'],
  'success_at_k': True,
  'top1_relevant': True},
 {'id': 'q_2',
  'query': 'How do I run a repeated measures ANOVA in JASP?',
  'answerable': True,
  'retrieved_ids': ['8638afc4', '75ac602b', '6192d3ce'],
  'success_at_k': True,
  'top1_relevant': False}]

In [60]:
#Step 5 — Save Retrieval Results
RETR_IEVAL_FILE = REPO_DIR / "data/test_QA/retrieval_results.jsonl"

with open(RETR_IEVAL_FILE, "w") as f:
    for r in retrieval_results:
        f.write(json.dumps(r) + "\n")

RETR_IEVAL_FILE


PosixPath('/Users/ywxiu/jasp-multimodal-rag/data/test_QA/retrieval_results.jsonl')

In [66]:


import pandas as pd
import json

# --- Step 1: Read the JSONL file ---
file_path = '/Users/ywxiu/jasp-multimodal-rag/data/test_QA/retrieval_result.jsonl'
retrieval_results = []

try:
    with open(file_path, 'r') as f:
        for line in f:
            # Each line in a JSONL file is a self-contained JSON object
            retrieval_results.append(json.loads(line))
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    # Handle the error, perhaps by exiting or returning an empty result
    # For this example, we'll continue with an empty list which will lead to a 0/NaN result.
    # In a real script, you might want to stop execution here.
    pass

In [67]:
#Step 6 — Compute H1a & H1b
df = pd.DataFrame(retrieval_results)

df_answerable = df[df["answerable"] == True]
df_unanswerable= df[df["answerable"] == False]
print(df_answerable)
print(df_unanswerable)

H1a = df_answerable["success_at_k"].mean()
H1b = df_answerable["top1_relevant"].mean()


is_empty_retrieval = (df_unanswerable["retrieved_ids"].str.len() == 0)
H3 = is_empty_retrieval.mean()

H1a, H1b,H3


    id                                              query  answerable  \
0  q_1    How to import data from Excel or CSV into JASP?        True   
1  q_2    How do I run a repeated measures ANOVA in JASP?        True   
3  q_4          How to run PAIRED SAMPLES T-TEST in JASP?        True   
4  q_5        Can JASP run Principal Component Analysis?         True   
5  q_6    How to know if my data meets ANOVA assumptions?        True   
6  q_7  How to perform a Bayesian regression and inter...        True   

                    retrieved_ids  success_at_k  top1_relevant  
0                              []         False          False  
1  [75ac602b, 8638afc4, 6192d3ce]          True           True  
3                      [4488c4f1]          True           True  
4                              []         False          False  
5                              []         False          False  
6                              []         False          False  
      id                         

(np.float64(0.3333333333333333),
 np.float64(0.3333333333333333),
 np.float64(1.0))

# PART B — GENERATION EVALUATION (H2a, H2b)

H2a. At least 90% of generated answers will receive a rating of “ambiguous” or “good,” indicating that the majority of system outputs are meaningfully related to the user’s query.
H2b. At least 60% of generated answers will receive a rating of “good.”


In [63]:


# --- 1. Define File Path and Test Queries ---
file_path = '/Users/ywxiu/jasp-multimodal-rag/data/test_QA/retrieval_results.jsonl'



# Open the file in 'append' mode ('a') to add new lines
# Using 'w' would overwrite the entire file each time.
try:
    with open(file_path, 'a') as outfile:
        for i, query in enumerate(query):
            print(f"Processing query {i+1}/{len(query)}: '{query[:50]}...'")

            # Call your generation function
            result_dict = run_generation_pipeline(query=query)

            # Convert the result dictionary to a JSON string
            json_line = json.dumps(result_dict)

            # Write the JSON string followed by a newline character to the file
            outfile.write(json_line + '\n')

    print(f"\n✅ Successfully saved results to: {file_path}")

except FileNotFoundError:
    # This handles cases where the directory structure might not exist
    print(f"\n❌ Error: The directory for {file_path} was not found.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

# --- 3. Verification (Optional) ---
# You can read the file back to confirm the content
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        line_count = sum(1 for line in f)
        print(f"File verification: {line_count} lines found in the file.")


❌ An unexpected error occurred: name 'query' is not defined
File verification: 12 lines found in the file.
